In [199]:
import numpy as np
import csv
import math

In [213]:
#xu li data
def loadData(path):
    f = open(path,'r')
    data = csv.reader(f)
    data = np.array(list(data))
    data = np.delete(data,0,0)
    np.random.shuffle(data)
    f.close()
    classes = []
    tp = []
    for i in range(len(data)):
      t = 0
      for j in range(len(data[i])):
        if(data[i][j]==''):
          t = t+1
      if(t!=0):
        tp.append(i)
    data = np.delete(data,[i for i in tp],0)
    for dt in data:
        if(dt[-1] not in classes):
            classes.append(dt[-1])
    for i in range(len(classes)):
      for j in range(len(data)):
        if(data[j][-1]==classes[i]):
          data[j][-1]=i
    Mdata = [[]*len(data[0])]*len(data)
    for i in range(len(data)):
      Mdata[i] = [float(x) for x in data[i]]
    trainSet = Mdata[:106]
    testSet = Mdata[106:]
    return trainSet,testSet

In [214]:
trainSet,testSet = loadData('Iris.csv')

In [215]:
#Nhom cac du lieu co cung nhan
def group(data):
  group_dict = {}
  for i in range(len(data)):
    if(data[i][-1] not in group_dict):
      group_dict[data[i][-1]] = []
    group_dict[data[i][-1]].append(data[i])
  return group_dict

In [216]:
#Ki vong
def Mean(Num):
  return sum(Num)/float(len(Num))

In [217]:
#Do lech chuan
def standardDeviation(numbers):
  avg = Mean(numbers)
  ve = sum([ pow(x-avg, 2) for x in numbers ])/float(len(numbers)- 1) #tinh phuong sai cua tung diem du lieu
  return math.sqrt(ve) #tra ve do lech chuan cua diem du lieu

In [218]:
def meanAndStandardDeviation(data):
  info = [( Mean(x),standardDeviation(x) ) for x in zip(*data)]
  del info[-1]
  return info

In [219]:
group_dict = group(trainSet)
def meanAndStandardDeviationForClass(group_dict):
  info = {}
  for classValue,instances in group_dict.items():
    info[classValue] = meanAndStandardDeviation(instances)
  return info

In [220]:
#ham tinh mat do xac suat Gauss
#P(x_i|c)(do day la phan phoi Gauss nen dung cong thuc nay)
def calGaussianProbility(x_i,mean,standardDeviation):
  ex = math.exp(- (pow(x_i-mean,2)/(2*pow(standardDeviation,2))))
  return (1/(math.sqrt(2*math.pi) * standardDeviation)) * ex

In [221]:
#ham tinh xac xuat cua cac nhan
#P(c|x)
def classesProbility(info, x):
  probilities = {}
  #tinh ki vong va do lech chuan cua tung class
  for classV,ClassSummaries in info.items():
    probilities[classV] = len(group_dict[classV])/float(len(trainSet))
    for i in range(len(ClassSummaries)):
      mean,standardDeviation = ClassSummaries[i]
      x_i = x[i]
      probilities[classV] *= calGaussianProbility(x_i,mean,standardDeviation)
  return probilities

In [222]:
#tim nhan co xac suat cao nhat
def predict(info,x):
  probilities = classesProbility(info,x)
  bestLabel,hightPro = None,-1
  for classValue,probality in probilities.items():
    if bestLabel is None or hightPro<probality:
      bestLabel = classValue
      hightPro = probality
  return bestLabel 

In [223]:
def getPredict(info,test):
  predictions = []
  for i in range(len(test)):
    predictions.append(predict(info,test[i]))
  return predictions

In [224]:
def doChinhXac(test,predictions):
  yes = 0
  for i in range(len(test)):
    if predictions[i]==test[i][-1]:
      yes = yes+1
  return (float(yes)/float(len(test)))*100.0

In [225]:
info = meanAndStandardDeviationForClass(group_dict)
predictions = getPredict(info,testSet)
doChinhXac(testSet,predictions)


95.45454545454545